In [156]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split

#Librerias para visualizacion
import seaborn as sb
import matplotlib.pyplot as plt

In [157]:
labels_training = pd.read_csv('/home/lucas/Documentos/Facultad/OrgaDeDatos/tp/labels_training_set.csv',
                              low_memory= False)
events_data = pd.read_csv('/home/lucas/Documentos/Facultad/OrgaDeDatos/tp/events_up_to_01062018.csv', 
                              low_memory= False)

In [158]:
#Data Frame de features
features = pd.DataFrame()

In [159]:
events_data.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [160]:
labels_training.head()

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [161]:
#En features tengo los usuarios sin repetir
features['person'] = events_data['person'].unique()

In [162]:
events_data['hour'] = pd.to_datetime(events_data['timestamp']).dt.hour
events_data['date'] = pd.to_datetime(events_data['timestamp']).dt.date

**Cantidad de conversiones por usuario**

In [163]:
conversions_user = pd.DataFrame()

In [164]:
conversions_user = events_data.loc[events_data['event'] == 'conversion',['person']]

In [165]:

conversions_user= pd.DataFrame(conversions_user['person'].value_counts())

In [166]:
conversions_user = conversions_user.reset_index()
conversions_user.rename({'index':'person', 'person':'cantidad de conversiones'}, axis = 1, inplace = True)
conversions_user.head()

,person,cantidad de conversiones
0,5059f7fd,129
1,888a11a2,44
2,0280a93d,32
3,b4cdf2b5,30
4,32c21f9e,21


In [167]:
features = features.merge(conversions_user, on = 'person', how = 'left')

In [168]:
features.head()

,person,cantidad de conversiones
0,4886f805,NaN
1,ad93850f,NaN
2,0297fc1e,NaN
3,2d681dd8,NaN
4,cccea85e,NaN


In [169]:
#features['cantidad de conversiones'] = features['cantidad de conversiones'].fillna(0) 

In [170]:
features.head(20)

,person,cantidad de conversiones
0,4886f805,NaN
1,ad93850f,NaN
2,0297fc1e,NaN
3,2d681dd8,NaN
4,cccea85e,NaN
5,4c8a8b93,NaN
6,1b9f7cf6,NaN
7,29ebb414,NaN
8,de8fe91b,NaN
9,45baf068,NaN


In [171]:
events_data['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

**Cantidad de veces que cada usuario vio un producto**

In [172]:
view_user = pd.DataFrame()
view_user = events_data.loc[events_data['event'] == 'viewed product',['person']]
view_user= pd.DataFrame(view_user['person'].value_counts())
view_user = view_user.reset_index()
view_user.rename({'index':'person', 'person':'cantidad de veces que vio un producto'}, axis = 1, inplace = True)
view_user.head()

,person,cantidad de veces que vio un producto
0,6abd2bf1,2355
1,b1f4dbf6,2233
2,8fb4929e,1912
3,06ed04d6,1881
4,bfb74b38,1782


In [173]:
features.head(10)

,person,cantidad de conversiones
0,4886f805,NaN
1,ad93850f,NaN
2,0297fc1e,NaN
3,2d681dd8,NaN
4,cccea85e,NaN
5,4c8a8b93,NaN
6,1b9f7cf6,NaN
7,29ebb414,NaN
8,de8fe91b,NaN
9,45baf068,NaN


In [174]:
features = features.merge(view_user, on = 'person', how = 'left')
#features['cantidad de veces que vio un producto'] = features['cantidad de veces que vio un producto'].fillna(0) 

In [175]:
features.head()

,person,cantidad de conversiones,cantidad de veces que vio un producto
0,4886f805,NaN,4.0
1,ad93850f,NaN,20.0
2,0297fc1e,NaN,404.0
3,2d681dd8,NaN,13.0
4,cccea85e,NaN,739.0


**Cantidad de visitas a la pagina de cada usuario**

In [176]:
visit_user = pd.DataFrame()
visit_user = events_data.loc[events_data['event'] == 'visited site',['person']]
visit_user= pd.DataFrame(visit_user['person'].value_counts())
visit_user = visit_user.reset_index()
visit_user.rename({'index':'person', 'person':'cantidad de visitas a la pagina'}, axis = 1, inplace = True)
visit_user.head()

,person,cantidad de visitas a la pagina
0,5059f7fd,295
1,02f14240,294
2,67bdc946,210
3,7ac0c607,208
4,9b3b43aa,184


In [177]:
features = features.merge(visit_user, on = 'person', how = 'left')
#features['cantidad de visitas a la pagina'] = features['cantidad de visitas a la pagina'].fillna(0)
features.head()

,person,cantidad de conversiones,cantidad de veces que vio un producto,cantidad de visitas a la pagina
0,4886f805,NaN,4.0,1.0
1,ad93850f,NaN,20.0,5.0
2,0297fc1e,NaN,404.0,95.0
3,2d681dd8,NaN,13.0,2.0
4,cccea85e,NaN,739.0,22.0


In [178]:
features['cantidad de veces que vio un producto'].isnull().any()

True

In [179]:
features['cantidad de visitas a la pagina'].isnull().any()

True

**Cantidad de visitas a la pagina de cada usuario**

In [180]:
check_user = pd.DataFrame()
check_user = events_data.loc[events_data['event'] == 'checkout',['person']]
check_user= pd.DataFrame(visit_user['person'].value_counts())
check_user = check_user.reset_index()
check_user.rename({'index':'person', 'person':'cantidad de checkouts'}, axis = 1, inplace = True)
check_user.head()

,person,cantidad de checkouts
0,37ab1db5,1
1,c91ae64f,1
2,0b6ef3d1,1
3,f164fbff,1
4,c8595b3a,1


In [181]:
features = features.merge(check_user, on = 'person', how = 'left')
#features['cantidad de checkouts'] = features['cantidad de checkouts'].fillna(0)
features.head()

,person,cantidad de conversiones,cantidad de veces que vio un producto,cantidad de visitas a la pagina,cantidad de checkouts
0,4886f805,NaN,4.0,1.0,1.0
1,ad93850f,NaN,20.0,5.0,1.0
2,0297fc1e,NaN,404.0,95.0,1.0
3,2d681dd8,NaN,13.0,2.0,1.0
4,cccea85e,NaN,739.0,22.0,1.0


**Veo cuantas veces vio cada uno de los 5 modelos mas vendidos segun lo obtenido en la exploracion del tp1**

iPhone 6S, Samsung Galaxy S6 Flat, iPhone 6 , iPhone 5S, Samsung Galaxy J5

Veo por usuario cuantas veces vio uno de estos productos